In [ ]:
# + проверено (взято новое)

# 1. Импорты и загрузка файлов
# 1.0. Импорты
import pandas as pd
import pathlib as pth
from datetime import datetime

data_dir = pth.Path(r'c:\DATA\DOC\_Personal\учеба\VS Code\44. Проект\data')
events_file = data_dir/r'7_4_Events.csv'
purchase_file = data_dir/r'7_4_Purchase.csv'

levels_arranged = ('easy', 'medium', 'hard')
events_arranged = ('registration', 'tutorial_start', 'tutorial_finish', 'level_choice', 'pack_choice', 'purchase')

# 1.1. Читаем основной лог
events_df = pd.read_csv ( events_file,
      dtype = {'selected_level': 'category',
              'id': 'Int64', 'tutorial_id': 'Int64', 'user_id': 'Int64'}
    )
events_df['start_time'] = pd.to_datetime \
        (   events_df['start_time'],
            format='%Y-%m-%dT%H:%M:%S', errors='coerce'
        )

# 1.2. Читаем файл про покупки
purchase_df = pd.read_csv ( purchase_file,
     dtype =  {'id': 'Int64', 'user_id': 'Int64',
               'amount': 'float64'}
    )
purchase_df['event_datetime'] = pd.to_datetime \
        (   purchase_df['event_datetime'],
            format='%Y-%m-%dT%H:%M:%S', errors='coerce'
        )

In [ ]:
# + проверено

# Функции
# Краткое описание датафрейма в удобной мне форме
def df_describe(df):
    return pd.DataFrame\
    ({  'dtype':        df.dtypes
        , 'distinct':   df.nunique()
        , 'empty':      df.isna().sum()
    })

# "Сплющивание" многоуровневых столбцов
def df_flatten_headers (df):
    df.columns = \
        [':'.join([x for x in col]) 
        for col in df.columns.to_flat_index()
        ]
    return df

In [ ]:
# 2. Характеристика данных.
# Непосредственно ход первичного анализа данных пропущу. 
# Ниже представлены его итоги

print ('events_df')
display (df_describe (events_df))
print ('purchase_df')
display (df_describe (purchase_df))
print ("purchase_df['amount']")
display (purchase_df['amount'].describe())

### X. Выводы по первичному анализу данных
#### events_df
1. Пропуски в столбцах selected_level, tutorial_id, что логично, т.к. не все пользователи выбирали уровень и проходили обучение
2. Пропуск в столбце start_time: в исходных данных были некорректные (2017.02.29), либо запорченные (20162015-09-18) даты. Решено было пропустить такие даты, не пытаясь интерпретировать
3. Столбец selected_level приводится к категориальному формату сразу.
4. Столбец event_type будет приведен к категориальному формату позже - после соединения датафреймов (будет добавлена категория "purchase")
5. Уникальных таймстампов немного меньше, чем id событий (даже с учетом пропусков)
#### purchase_df
1. Пропусков нет
2. id покупок столько же, сколько пользователей
#### Сумма покупки
1. Поле **purchase_df['**amount**']** выглядит как обычная сумма покупки, аномалий не прослеживается

In [ ]:
# 3. Фильтрация и соединение исходных данных
# 3.1. Формируем выборку пользователей по условию задания
filtering_sample = ( events_df
    .query (f"event_type=='registration' & start_time.dt.year==2018")
    ['user_id'].unique ()
)

# 3.2 Фильтруем и слегка подобрабатываем датафреймы
events_df_filtered = ( events_df
    .query (f"user_id in @filtering_sample")
    .rename (columns={'id': 'event_id'})
)

purchase_df_filtered = ( purchase_df
    .query (f"user_id in @filtering_sample")
    .rename (columns={'id': 'purchase_id', 'event_datetime': 'start_time'})
    .assign (**{'event_type': 'purchase'})
)

# 3.3 Соединяем. Вот с этим мы будем работать
events_combined = \
(   pd.concat( [events_df_filtered, purchase_df_filtered], sort=False )
    .reset_index (drop=True)
    .sort_values (by='start_time')
    .astype
    ({  'selected_level':   pd.api.types.CategoricalDtype(categories=levels_arranged, ordered=True)
        , 'event_type':     pd.api.types.CategoricalDtype(categories=events_arranged, ordered=True)
    })
)

# Получился датафрейм следующего формата:
print ('events_combined')
display (df_describe (events_combined))

In [ ]:
# Исследуем структуру данных по событиям
# 1. Посчитаем количество событий каждого вида в жизни пользователя
event_times_per_user = \
(events_combined.pivot_table
    (   index = 'user_id'
        , columns = 'event_type'
        , values = 'start_time'
        , aggfunc = 'count'
    )
)
# 2. Теперь рассмотрим результаты
event_times_per_user.describe()

### Распределение событий в жизни пользователя
1. Регистрация происходит ровно 1 раз (не зарегистрировавшиеся пользователи в статистику просто не попали)
2. level_choice, pack_choice, purchase происходят 0 или 1 раз, т.е. либо могут не происходить, но если происходят, то 1 раз *(что странно)*
3. Среднее по ним при этом убывает, т.к. каждое следующее событие совершает меньшее кол-во пользователей
4. У tutorial_start, tutorial_finish максимум 9 (т.е. кто-то максимум 9 раз начинал и заканчивал обучение), 
5. Но средние различаются, доля закончивших обучение меньше

In [ ]:
# Сравним статистику по началам и завершениям обучений.
start_column = event_times_per_user['tutorial_start'].value_counts()
finish_column = event_times_per_user['tutorial_finish'].value_counts()

tutorials_per_user = pd.DataFrame \
    ({  'start': start_column
        , 'finish': finish_column
        , 'start, %': (start_column / start_column.sum()).round(3)*100
        , 'finish, %': (finish_column / finish_column.sum()).round(3)*100
    })
tutorials_per_user.index.rename('times', inplace=True)
display (tutorials_per_user)

### Начала и зввершения обучений
* 40% пользователей не начинали обучение, 46% начинали 1 раз, 8% начинали 2 раза. 
* Далее (до 9 раз) идут не очень существенные в процентном отношении, но заметные в абсолютном количества (десятки и сотни людей)
* 49% пользователей не закончили обучение. 40% закончили 1 раз, 7% закончили 2 раза
* Далее - картина аналогична начавшим обучение (но в среднем чуть меньше для каждого кол-ва раз)

In [ ]:
# Подготовка к заполнению основной таблицы
max_event_per_user = \
(events_combined
    .groupby (['user_id'],as_index=False)
    ['event_type'].max()
    .rename (columns={'event_type':'last_event'})
)

In [ ]:
# Формируем основную таблищу, с которой будем раборать

users_event_data = \
(events_combined
    .groupby (['user_id', 'event_type'])
    ['start_time']
    .agg(['count','min','max'])
    .rename (columns={'min': 'first', 'max': 'last'})
    .assign
    (**{  'first_dif': lambda df: df.groupby(['user_id'])['first'].diff()
        , 'last_dif': lambda df: df.groupby(['user_id'])['last'].diff()
    })
    .reset_index()
    .merge (max_event_per_user, left_on='user_id',right_on='user_id')
    .set_index(['user_id', 'event_type'])
)
users_event_data.head (50)

In [ ]:
# Пробуем отвечать на вопрос №1
# Отличается ли время прохождения различных этапов у пользователей, которые прошли обучение, и пользователей, не начинавших обучение
(users_event_data
    .query 
    (f"last_event>'tutorial_finish'"
     f"and event_type == 'tutorial_start'"
    )
    ['count'].value_counts()
    
)

In [ ]:
users_event_data_wide = \
(events_combined
    .groupby (['user_id', 'event_type'])
    ['start_time']
    .agg(['count','min'])
    .rename (columns={'min': 'first'})
    .assign
    (**{  'first_dif': lambda df: df.groupby(['user_id'])['first'].diff()
    })
    .reset_index()
    .merge (max_event_per_user, left_on='user_id',right_on='user_id')
    .pivot_table
    (   index=['user_id', 'last_event']
        , columns='event_type'
        , values=['count', 'first', 'first_dif']
        , aggfunc= lambda x: x
    )
    .apply (flatten_headers)
)
users_event_data_wide.head (20)

In [ ]:
# Изучаем сплющивание имен столбцов
# 0. Смотрим, что есть. Есть MultiIndex, состоящий из кортежей
users_event_data_wide.columns
# MultiIndex([(    'count',    'registration'),
#             (    'count',  'tutorial_start'),
#             ...
# 1. Плющим - этап 1. Получаем "плоский" индекс. Его элементы - все те же кортежи, но он уже плоский, не разбиваемый на уровни
users_event_data_wide.columns.to_flat_index()
# Index([       ('count', 'registration'),      ('count', 'tutorial_start'), ...
# 2. Плющим - этап 2 - из этих кортежей получаем моностроковые имена. В виде списка.
[':'.join([x for x in col]) 
 for col in users_event_data_wide.columns.to_flat_index()
]
# 3. А теперь внимание - пробуем наложить этот наш список на исходный индекс столбцов
users_event_data_wide.columns = \
[':'.join([x for x in col]) 
 for col in users_event_data_wide.columns.to_flat_index()
]
users_event_data_wide
# Сработало!




In [ ]:
(users_event_data_wide
    .apply (lambda df:
        df.rename
        ([':'.join([x for x in col]) for col in df.columns.to_flat_index()], axis='columns'
         )
    )
)

In [ ]:
# Как работает query с многоуровневыми столбцами:

(users_event_data_wide
    .query ("`('count', 'tutorial_start')`>1")
)


# [['user_id', 'last_event']]